In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation,Dropout,BatchNormalization
from tensorflow.keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D,Bidirectional,LSTM
from tensorflow.keras.optimizers import Adam

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import time
import json

In [3]:
train_json=r'E:\iiit\Sem3\Information Extraction And Retrieval\major_project\data\json\train_data.json'
val_json=r'E:\iiit\Sem3\Information Extraction And Retrieval\major_project\data\json\val_data.json'

In [4]:
df_train=pd.read_json(train_json,orient='table')

In [7]:
embeddings=hub.load(r'.\USE4')

# save train embeddings

In [7]:
train_embeddings=[]

In [8]:
for text in tqdm(df_train['clean_text'].values.tolist()):
    vector=tf.make_ndarray(tf.make_tensor_proto(embeddings([text]))).tolist()
    train_embeddings.append(vector)

100%|███████████████████████████████████████████████████████████████████████| 600000/600000 [1:19:37<00:00, 125.60it/s]


In [11]:
np.save('use_embeddings_large.npy',train_embeddings)

# save val embeddings

In [12]:
df_test=pd.read_json(val_json,orient='table')

In [13]:
test_embeddings=[]

In [14]:
for text in tqdm(df_test['clean_text'].values.tolist()):
    vector=tf.make_ndarray(tf.make_tensor_proto(embeddings([text]))).tolist()
    test_embeddings.append(vector)

100%|█████████████████████████████████████████████████████████████████████████| 150000/150000 [22:19<00:00, 111.97it/s]


In [15]:
np.save('use_embeddings_small.npy',test_embeddings)

# load saved  train embeddings and labels

In [3]:
x=np.load('use_embeddings_large.npy')

In [4]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
labels=np.load('hyperpartisan_large.npy',allow_pickle=True)
le=LabelEncoder()
le_labels=le.fit_transform(labels)
# ohe=OneHotEncoder()
# ohe_labels=ohe.fit_transform(le_labels.reshape(-1,1))

In [5]:
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

In [8]:
model = Sequential()
model.add(Dense(512,activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation = 'tanh'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
input_shape=(None,512)

In [10]:
model.build(input_shape) # `input_shape` is the shape of the input data
                         # e.g. input_shape = (None, 32, 32, 3)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  262656    
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  131328    
_________________________________________________________________
batch_normalization (BatchNo multiple                  1024      
_________________________________________________________________
dense_2 (Dense)              multiple                  32896     
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  8

In [11]:
history=model.fit(x.reshape(-1,512),le_labels,epochs=5,batch_size=32,shuffle=True,validation_split=0.2)

Epoch 1/5
15000/15000 [==============================] - 56s 4ms/step - loss: 0.4132 - accuracy: 0.8050 - val_loss: 0.3632 - val_accuracy: 0.8336
Epoch 2/5
15000/15000 [==============================] - 58s 4ms/step - loss: 0.3682 - accuracy: 0.8307 - val_loss: 0.3459 - val_accuracy: 0.8447
Epoch 3/5
15000/15000 [==============================] - 56s 4ms/step - loss: 0.3518 - accuracy: 0.8397 - val_loss: 0.3372 - val_accuracy: 0.8478
Epoch 4/5
15000/15000 [==============================] - 55s 4ms/step - loss: 0.3400 - accuracy: 0.8453 - val_loss: 0.3307 - val_accuracy: 0.8523
Epoch 5/5
15000/15000 [==============================] - 56s 4ms/step - loss: 0.3320 - accuracy: 0.8504 - val_loss: 0.3270 - val_accuracy: 0.8532


# load test data

In [12]:
x_test=np.load('use_embeddings_small.npy')

In [13]:
x_test.shape

(150000, 1, 512)

In [14]:
predictions=model.predict_classes(x_test.reshape(-1,512))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [15]:
np.unique(predictions,return_counts=True)

(array([0, 1], dtype=int64), array([ 40474, 109526], dtype=int64))

In [16]:
preds=le.inverse_transform(predictions)

In [17]:
actuals=np.load('hyperpartisan_small.npy',allow_pickle=True)

In [18]:
from sklearn.metrics import accuracy_score,f1_score

In [19]:
accuracy_score(actuals,preds)

0.5819066666666667

In [20]:
f1_score(le.transform(actuals),le.transform(preds))

0.6601346151761811